In [7]:
import numpy as np
import pandas as pd
import json

In [8]:
base = "kyungmin_cutflows_15Oct24/"

In [32]:
df1 = pd.read_csv(base+"SR_Signal_ctau-1_samples_Raw.csv")
df2 = pd.read_csv(base+"SR_Signal_ctau-10_samples_Raw.csv")
df3 = pd.read_csv(base+"SR_Signal_ctau-100_samples_Raw.csv")

df1['name'] = df1['Unnamed: 0']
df2['name'] = df2['Unnamed: 0']
df3['name'] = df3['Unnamed: 0']

df = pd.concat([df1,df2,df3])

In [53]:
with open("/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMe/python_analysis/configs/sample_configs/signal/signal_2018_v12_miniAOD_aEM.json","r") as fin:
    ref = json.load(fin)

In [57]:
nevts = {r['name'].replace(".","p").replace("ct-","ctau-").replace("sig_","sig_2018_"):r['num_events'] for r in ref}
oldNames = {r['name'].replace(".","p").replace("ct-","ctau-").replace("sig_","sig_2018_"):r['name'] for r in ref}

In [82]:
nevts_needed = {}
for name,tot in nevts.items():
    remain = df[df.name==name].BDT.to_numpy()[0]
    ntarg = 1000
    eff = remain/tot
    needed = int(ntarg/eff)
    nevts_needed[name] = np.round(needed,-4)
    
    print(f"{name}: {remain}/{tot} ({100*remain/tot:.4f}%), need {needed} --> {np.round(needed,-4)}")
    mchi = float(name.split("_")[2].split("-")[1].replace("p","."))
    dmchi = float(name.split("_")[3].split("-")[1].replace("p","."))
    ct = mchi = int(name.split("_")[4].split("-")[1])

sig_2018_Mchi-105p0_dMchi-10p0_ctau-1: 1342/78615 (1.7071%), need 58580 --> 60000
sig_2018_Mchi-105p0_dMchi-10p0_ctau-10: 10702/631085 (1.6958%), need 58968 --> 60000
sig_2018_Mchi-105p0_dMchi-10p0_ctau-100: 4444/778905 (0.5705%), need 175271 --> 180000
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1: 1491/513160 (0.2906%), need 344171 --> 340000
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10: 948/643295 (0.1474%), need 678581 --> 680000
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100: 367/1437170 (0.0255%), need 3915994 --> 3920000
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1: 4533/257674 (1.7592%), need 56844 --> 60000
sig_2018_Mchi-110p0_dMchi-20p0_ctau-10: 5284/267815 (1.9730%), need 50684 --> 50000
sig_2018_Mchi-110p0_dMchi-20p0_ctau-100: 1810/258432 (0.7004%), need 142780 --> 140000
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1: 1799/443760 (0.4054%), need 246670 --> 250000
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10: 1212/559380 (0.2167%), need 461534 --> 460000
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100: 279/732332 (0.0381%), need 262484

In [83]:
with open("request.csv","r") as fin:
    request = fin.read().splitlines()


In [96]:
top = request[0]
outlines = []
for line in request[1:]:
    info = "_".join(line.split(",")[0].split("_")[2:5])
    match = False
    for k in nevts_needed.keys():
        b = "_".join(k.split("_")[2:])
        if b == info:
            match = True
            n = nevts_needed[k]
            if n > 2_000_000:
                n = 2_000_000
            break
    if match:
        outlines.append(line.replace("50000",str(n)))
    else:
        print(info,'NO MATCH')
        outlines.append(line.replace("50000","XXXXX"))

Mchi-66p0_dMchi-12p0_ctau-1 NO MATCH
Mchi-66p0_dMchi-12p0_ctau-10 NO MATCH
Mchi-66p0_dMchi-12p0_ctau-100 NO MATCH
Mchi-88p0_dMchi-16p0_ctau-1 NO MATCH


In [98]:
with open("request2.csv","w") as fout:
    fout.write("\n".join([top]+outlines))

In [89]:
request[1]

'iDM_DarkPhotonToEE_Mchi-5p25_dMchi-0p5_ctau-1_mA-15p0_TuneCP5_13TeV_madgraph-pythia8,genFragments/Hadronizer/13TeV/iDM/iDMe_pythiaGenFragment_ctau-1.py,50000,madgraph,/afs/cern.ch/user/s/sabright/public/iDMe/gridpacks/iDMe_Mchi-5p25_dMchi-0p5_mZDinput-15p0_ctau-0_1jet_icckw1_drjj0_xptj80_xqcut20_slc7_amd64_gcc820_CMSSW_10_6_28_tarball.tar.xz'

In [ ]:
m = [50,60,70]
nev = [80000,None,70000]

In [102]:
np.interp(80,[70,90],[70000,60000])

65000.0